In [1]:
import plotly.io as pio
from urllib.request import urlopen
import json
from sqlalchemy import create_engine
import psycopg2
import plotly.express as px
import pandas as pd

# Chloropleth Example

In [2]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})



fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )

pio.write_json(fig, '/home/acc/output/chloro.plotly')

In [3]:
# Create HTML

In [4]:
with open('/home/acc/output/chloro.html', 'w') as writer:
        writer.write(fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}).to_html())

# Scatter Plot Example

In [5]:
url = "https://www.dropbox.com/s/15gisj8hx218rn1/street-pole-sample.csv?dl=1"
df = pd.read_csv(url)
df.head()

,Unnamed: 0,X,Y,OBJECTID,OID,POLE_NUM,TYPE,NLUMIN,LUM_SIZE,HEIGHT,POLE_DATE,UP_DATE,OWNER,TAP_ID,BLOCK,PLATE
0,423,-75.170097,39.942766,41341,0,214423,WP,NaN,NaN,NaN,1997-06-09T00:00:00.000Z,1997-06-09T00:00:00.000Z,PECO,0.0,NaN,NaN
1,85,-75.166112,39.941477,42523,0,215645,AAPT,2.0,250.0,25.0,1997-06-10T00:00:00.000Z,1997-06-10T00:00:00.000Z,Streets,214622.0,NaN,NaN
2,121,-75.163483,39.943068,42730,0,215926,WP,NaN,NaN,NaN,1997-06-04T00:00:00.000Z,1997-06-04T00:00:00.000Z,PECO,0.0,NaN,NaN
3,478,-75.167727,39.944528,41409,0,214515,WP,1.0,100.0,25.0,1997-06-09T00:00:00.000Z,1997-06-09T00:00:00.000Z,Streets,214515.0,NaN,NaN
4,392,-75.171863,39.941486,41298,0,214380,WP,1.0,100.0,25.0,1997-06-09T00:00:00.000Z,1997-06-09T00:00:00.000Z,Streets,214380.0,NaN,NaN


In [6]:
df['TEXT'] = 'hi'

In [7]:
fig = px.scatter_mapbox(df, lat="Y", lon="X", hover_data=["TEXT"], mapbox_style="carto-positron", zoom=15)
with open('/home/acc/output/scatter.html', 'w') as writer:
        writer.write(fig.to_html())

In [8]:
pio.write_json(fig, '/home/acc/output/scatter.plotly')

# Twitter Example

In [51]:
# sql = '''select text, lat, lon, full_name, sentiment  from tweets_transform where lower(text) like '%{0}%';'''.format('biden')
sql = '''select text, lat, lon, full_name, sentiment  from tweets_transform'''
conn = psycopg2.connect("dbname='twitter' user='postgres' host='10.0.0.26' port = '5432' password='password'")
cur = conn.cursor()
cur.execute(sql)
df = pd.DataFrame(cur.fetchall())
df.columns = ['text', 'lat', 'lon', 'full_name', 'sentiment']
df['sentiment'] = (df['sentiment'] - 5)/10
df = df.dropna()
df.lat = df.lat.astype('float')
df.lon = df.lon.astype('float')

In [52]:
DF = df.groupby('full_name').mean()
DF['n'] = df.groupby('full_name').count()['sentiment'].values
DF = DF.sort_values('n', ascending = False).reset_index()
DF = DF[DF['n'] > 100]

In [55]:
fig = px.scatter_mapbox(DF, lat="lat", lon="lon",  hover_data=["full_name"], mapbox_style="carto-positron", color="sentiment", size = "n", zoom = 3.5, opacity = .5)
with open('/home/acc/output/tweet1.html', 'w') as writer:
        writer.write(fig.to_html())

In [56]:
pio.write_json(fig, '/home/acc/output/tweet1.plotly')

In [57]:
fig = px.scatter_mapbox(df, lat="lat", lon="lon",  hover_data=["text", "full_name", "sentiment"], mapbox_style="carto-positron", color="sentiment", zoom = 3.5, opacity = 1)
with open('/home/acc/output/tweet2.html', 'w') as writer:
        writer.write(fig.to_html())

In [58]:
pio.write_json(fig, '/home/acc/output/tweet2.plotly')